This is a solution to the "test your knowledge" challenges at the end of chapter 3. Be sure to try the challenges first before looking at the solutions! 

## Bitcoin price data

In [54]:
# ensure the db file is deleted so we are able to create it
import os
os.remove('bitcoin_data.sql')

In [55]:
# load json data
import json
with open('bitcoin_price.json') as f:
    data = json.load(f)

data[0]

{'symbol': 'btcusd',
 'time': 1364688000000,
 'open': 92.5,
 'close': 93.033,
 'high': 93.74999,
 'low': 91.0,
 'volume': 3083.07979075}

In [56]:
# we can see the data is organized in a list of dictionaries
data[0]['symbol']

'btcusd'

In [57]:
import sqlite3

In [58]:
db = sqlite3.connect('bitcoin_data.sql')
cur = db.cursor()

In [59]:
cur.execute('CREATE TABLE bitcoin_price (time INTEGER, open REAL, close REAL, high REAL, low REAL, volume REAL);')

In [60]:
data_list = []
for d in data:
    data_list.append([d['time'], d['open'], d['close'], d['high'], d['low'], d['volume']])

In [61]:
data_list[0]

[1364688000000, 92.5, 93.033, 93.74999, 91.0, 3083.07979075]

In [62]:
cur.executemany('INSERT INTO bitcoin_price VALUES (?, ?, ?, ?, ?, ?)', data_list)

In [63]:
db.commit()

In [64]:
db.close()

In [65]:
# re-connect to check data was properly saved
db = sqlite3.connect('bitcoin_data.sql')
cur = db.cursor()

In [66]:
# get latest date in the data
res = cur.execute('SELECT MAX(time) FROM bitcoin_price;')
max_time = res.fetchone()[0]
max_time

1606521600000

In [67]:
import datetime
datetime.datetime.fromtimestamp(max_time // 1000)

datetime.datetime(2020, 11, 27, 17, 0)

In [68]:
# get earliest date
res = cur.execute('SELECT MIN(time) FROM bitcoin_price;')
min_time = res.fetchone()[0]
datetime.datetime.fromtimestamp(min_time // 1000)

datetime.datetime(2013, 3, 30, 18, 0)

In [71]:
# get maximum price for each year, grouped and sorted by year
# first we need to create a year column
query = """
SELECT MAX(close), STRFTIME('%Y', DATE(ROUND(time / 1000), 'unixepoch')) as year
FROM bitcoin_price
GROUP BY year
ORDER BY year;
"""
res = cur.execute(query)
res.fetchall()

[(1138.5, '2013'),
 (919.24, '2014'),
 (466.01, '2015'),
 (981.7, '2016'),
 (19210.0, '2017'),
 (17169.0, '2018'),
 (12876.0, '2019'),
 (19131.0, '2020')]

In [72]:
db.close()

## chinook/sqlalchemy

In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///../chinook.db")
conn = engine.connect()

Get genres with the longest song length.

In [2]:
query = """
SELECT AVG(tracks.Milliseconds) / 1000, genres.Name
FROM tracks
JOIN genres
ON tracks.genreid = genres.genreid
GROUP BY tracks.genreid
ORDER BY AVG(Tracks.Milliseconds) DESC;
"""
res = conn.execute(query)
list(res)

[(2911.7830384615386, 'Sci Fi & Fantasy'),
 (2625.549076923077, 'Science Fiction'),
 (2575.28378125, 'Drama'),
 (2145.041021505376, 'TV Shows'),
 (1585.263705882353, 'Comedy'),
 (309.74944385026737, 'Metal'),
 (302.9858, 'Electronica/Dance'),
 (297.4529285714286, 'Heavy Metal'),
 (293.86756756756756, 'Classical'),
 (291.75537692307694, 'Jazz'),
 (283.9100431765613, 'Rock'),
 (270.35977777777777, 'Blues'),
 (264.05852500000003, 'Alternative'),
 (247.17775862068964, 'Reggae'),
 (244.37088372093024, 'Soundtrack'),
 (234.35384939759035, 'Alternative & Punk'),
 (232.85926252158893, 'Latin'),
 (229.03410416666665, 'Pop'),
 (224.92382142857142, 'World'),
 (220.0668524590164, 'R&B/Soul'),
 (219.59, 'Bossa Nova'),
 (189.16420833333333, 'Easy Listening'),
 (178.1762857142857, 'Hip Hop/Rap'),
 (174.813, 'Opera'),
 (134.6435, 'Rock And Roll')]

We can see thta the longest genres seem to actually be audio books, TV shows, and comedians.

Get the total sales for each year and genre.

In [9]:
query = """
SELECT SUM(invoices.total), STRFTIME('%Y', invoices.InvoiceDate) as year, genres.name
FROM invoices
JOIN invoice_items
ON invoices.invoiceid = invoice_items.invoiceid
JOIN tracks
ON invoice_items.trackid = tracks.trackid
JOIN genres
ON tracks.genreid = genres.genreid
GROUP BY year, tracks.genreid
ORDER BY year;
"""
res = conn.execute(query)
res_list = list(res)
res_list

[(1561.2300000000005, '2009', 'Rock'),
 (185.13, '2009', 'Jazz'),
 (502.92000000000013, '2009', 'Metal'),
 (466.29000000000025, '2009', 'Alternative & Punk'),
 (13.86, '2009', 'Rock And Roll'),
 (112.85999999999999, '2009', 'Blues'),
 (789.0300000000002, '2009', 'Latin'),
 (58.41, '2009', 'Reggae'),
 (13.86, '2009', 'Pop'),
 (45.53999999999999, '2009', 'Soundtrack'),
 (13.86, '2009', 'Bossa Nova'),
 (41.58, '2009', 'Easy Listening'),
 (50.489999999999995, '2009', 'Heavy Metal'),
 (69.3, '2009', 'R&B/Soul'),
 (17.82, '2009', 'Electronica/Dance'),
 (41.58, '2009', 'World'),
 (27.72, '2009', 'Hip Hop/Rap'),
 (1364.7700000000013, '2010', 'Rock'),
 (173.24999999999997, '2010', 'Jazz'),
 (488.2300000000002, '2010', 'Metal'),
 (264.37999999999994, '2010', 'Alternative & Punk'),
 (27.72, '2010', 'Rock And Roll'),
 (44.550000000000004, '2010', 'Blues'),
 (804.1400000000006, '2010', 'Latin'),
 (58.41, '2010', 'Reggae'),
 (61.38, '2010', 'Pop'),
 (55.44, '2010', 'Soundtrack'),
 (27.72, '2010', 'B

This is a ton of data, and we will have to use pandas and matplotlib to understand what it is telling us in the next chapter.

Below is a preview of the next chapter -- using pandas DataFrames and writing data to files:

In [13]:
import pandas as pd
df = pd.DataFrame(res_list, columns=['sales_USD', 'year', 'genre'])
df.to_csv('chinook_data.csv', index=False)
df.to_excel('chinook_data.xlsx', index=False)